<a href="https://colab.research.google.com/github/mvm1964/MGVB/blob/main/ann_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using {device} device")

Using cuda device


In [10]:
# prompt: create a torch neural net class with two hided layers and a single continuous node in the output layer

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, 1) # Single continuous output node
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [11]:
# prompt: create pandas dataframe with 20 continuous columns and 1000 rows filled with random numbers from 0 to 1

import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

# Create a DataFrame with 20 continuous columns and 1000 rows
#data = np.random.rand(1000, 20)
#df = pd.DataFrame(data)




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
df = pd.read_csv('/content/drive/My Drive/CUDA/data_for_GP.csv')

df

,barcode,TP53_genotype,TERT_expression,TP53_expression,AURKB_expression,PIK3CA_genotype,MCM7_expression,CDT1_expression,WDR62_expression,DDX12P_expression,FANCA_expression,CCPG1_expression,ELAC2_expression,Subtype,chr17p_loss
0,TCGA-A1-A0SB,wt,2.777472,12.113931,6.294367,wt,11.817677,8.109863,6.443701,6.031697,7.773672,10.483088,10.667074,1,0
1,TCGA-A1-A0SE,wt,0.410450,10.480347,6.695908,wt,11.070551,7.688933,7.791068,5.303879,6.668072,11.095231,9.825949,0,1
2,TCGA-A1-A0SF,wt,1.999567,10.761224,6.661727,wt,11.215446,8.339923,7.708767,6.364515,7.044879,10.342294,10.209237,0,1
3,TCGA-A1-A0SG,wt,1.286881,10.662042,6.252491,wt,10.256026,7.004965,7.602305,4.965336,6.107719,10.948345,10.531630,0,0
4,TCGA-A1-A0SH,wt,0.000000,10.263598,5.227910,wt,10.007178,6.004540,6.106363,4.634367,6.050151,11.748417,9.771334,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,TCGA-OL-A66J,wt,1.233888,10.659143,6.770258,wt,11.099064,7.079962,8.074620,6.294849,6.928022,11.168627,10.647435,0,0
663,TCGA-OL-A66K,wt,0.500292,10.790157,6.506826,wt,10.837435,6.614486,6.632678,5.952480,6.452694,10.941214,10.722052,0,0
664,TCGA-PE-A5DC,wt,2.516393,10.570715,8.942631,wt,11.495017,8.691711,9.215838,8.189915,7.752747,10.763663,10.033867,0,1
665,TCGA-PE-A5DD,wt,2.452385,10.653153,7.409273,mutant,11.078412,8.407137,8.097310,7.736244,7.926747,9.833109,10.054099,0,1


In [43]:
model = NeuralNetwork(input_size=7, hidden_size1=4, hidden_size2=4).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=7, out_features=4, bias=True)
    (1): ReLU()
    (2): Linear(in_features=4, out_features=4, bias=True)
    (3): ReLU()
    (4): Linear(in_features=4, out_features=1, bias=True)
  )
)


In [44]:

# Read data, convert to NumPy arrays
X = df.iloc[:, [4,6,7,8,9,10,11]].values
y = df.iloc[:, 3].values

#device = torch.device("cpu")

# convert into PyTorch tensors
X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device).reshape(-1, 1)

# create DataLoader, then take one batch
loader = DataLoader(list(zip(X,y)), shuffle=True, batch_size=7)



In [45]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train-test split for evaluation of the model
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

# set up DataLoader for training set
loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)


In [47]:
# train
n_epochs = 300
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()
for epoch in range(n_epochs):
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# evaluate accuracy after training
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))

Model accuracy: 0.00%
